Twitter_Sentimental_Analysis

In [1]:
import os
import json

from zipfile import ZipFile

import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shivaram/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
kaggle_dictionary=json.load(open("kaggle.json"))

In [4]:
os.environ["KAGGLE_USERNAME"]= kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"]= kaggle_dictionary["key"]

In [5]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [16]:
!ls

CIFAR.ipynb
IMDB Dataset.csv
IMDB_Sentimental_Analysis.ipynb
Logistic_Regression_mnist.ipynb
README.md
Untitled.ipynb
gan_generated_epoch_0.png
imdb-dataset-of-50k-movie-reviews.zip
kaggle.json
sentiment140.zip
training.1600000.processed.noemoticon.csv
venv_cnn


In [6]:
with ZipFile("sentiment140.zip","r") as zip_ref:
    zip_ref.extractall()

In [7]:
column_names=['target','ids','date','flag','user','text']

In [8]:
data=pd.read_csv("training.1600000.processed.noemoticon.csv",names=column_names ,encoding='ISO-8859-1')

In [9]:
data.shape

(1600000, 6)

In [10]:
data.isnull().sum()

target    0
ids       0
date      0
flag      0
user      0
text      0
dtype: int64

In [11]:
data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [12]:
data.replace({'target':{4:1}},inplace=True)

In [13]:
port_stem=PorterStemmer()

In [14]:
def stemming(content):
    stemmed_content=re.sub('[^a-zA-Z]',' ',content)
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split()
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content=' '.join(stemmed_content)

    return stemmed_content

In [15]:
data['stemmed_content']=data['text'].apply(stemming)

In [16]:
data

,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see
...,...,...,...,...,...,...,...
1599995,1,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...,woke school best feel ever
1599996,1,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...,thewdb com cool hear old walt interview http b...
1599997,1,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...,readi mojo makeov ask detail
1599998,1,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...,happi th birthday boo alll time tupac amaru sh...


In [17]:
x_data=data['stemmed_content'].values
y_label=data['target'].values

In [18]:
x_train,x_test,y_train,y_test=train_test_split(x_data,y_label,test_size=0.25,stratify=y_label,random_state=43)

In [19]:
vectorizer=TfidfVectorizer()

x_train=vectorizer.fit_transform(x_train)
x_test=vectorizer.transform(x_test)

In [20]:
x_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 8861378 stored elements and shape (1200000, 442348)>

In [21]:
x_test

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2859414 stored elements and shape (400000, 442348)>

In [22]:
model=LogisticRegression(max_iter=1000)

In [23]:
history=model.fit(x_train,y_train)

In [24]:
prediction=model.predict(x_test)
accuracy=accuracy_score(y_test,prediction)

In [25]:
print(f'Accuracy: {accuracy}')

Accuracy: 0.7769225


In [26]:
import pickle

In [27]:
file='trained_model.sav'
pickle.dump(model, open(file,'wb'))

In [28]:
loaded_model=pickle.load(open('trained_model.sav','rb'))

In [29]:
prediction=model.predict(x_test[200])
print(prediction)

print('positive' if prediction[0]==1 else 'negative')

[0]
negative


In [30]:
print(f'x_train:{x_train.shape} y_train:{y_train.shape} x_test:{x_test.shape} y_test{y_test.shape}')

x_train:(1200000, 442348) y_train:(1200000,) x_test:(400000, 442348) y_test(400000,)
